In [24]:
import pandas as pd
from functools import reduce
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

C:\Users\jeffr\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\jeffr\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
studentVLE=pd.read_csv("C:/Users/jeffr/Downloads/anonymisedData/studentVLE.csv")
studentAssessment=pd.read_csv("C:/Users/jeffr/Downloads/anonymisedData/studentAssessment.csv")
studentInfo=pd.read_csv("C:/Users/jeffr/Downloads/anonymisedData/studentInfo.csv")
studentRegistration=pd.read_csv("C:/Users/jeffr/Downloads/anonymisedData/studentRegistration.csv")

In [3]:
studentAssessment.dropna(inplace=True)
studentInfo.dropna(inplace=True)
studentRegistration.dropna(inplace=True)
studentVLE.dropna(inplace=True)

In [5]:
combined = [studentAssessment, studentInfo, studentRegistration, studentVLE]
df_combined = reduce(lambda left,right: pd.merge(left,right,on='id_student'), combined)
df_combined['final_result'].value_counts()

Withdrawn      16697024
Pass            6924889
Fail            2169591
Distinction     1149815
Name: final_result, dtype: int64

In [7]:
test2=df_combined.groupby('id_student')[['sum_click']].sum()

In [8]:
table = [studentInfo, test2]
table2 = reduce(lambda left,right: pd.merge(left,right,on='id_student'), table)
table2['final_result'].value_counts()

Withdrawn      5295
Pass            784
Fail            557
Distinction     119
Name: final_result, dtype: int64

In [11]:
le = preprocessing.LabelEncoder()
df_final = table2.apply(le.fit_transform)

In [12]:
df_final.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,sum_click
0,0,1,59,0,0,0,7,0,0,4,0,3,885
1,0,3,59,0,0,0,7,0,1,4,0,1,885
2,0,1,102,1,8,2,7,1,0,4,0,3,2236
3,0,3,102,1,8,2,7,1,1,4,0,2,2236
4,0,1,116,1,8,1,8,1,0,4,0,3,880


In [13]:
is1=df_final['code_presentation']==1

In [14]:
print(is1.head())

0     True
1    False
2     True
3    False
4     True
Name: code_presentation, dtype: bool


In [15]:
df_final1=df_final[is1]

In [16]:
is2=df_final['code_presentation']==2
is3=df_final['code_presentation']==3
is4=df_final['code_presentation']==4

In [17]:
df_final2=df_final[is2]
df_final3=df_final[is3]
df_final4=df_final[is4]

In [18]:
proj=pd.concat([df_final1,df_final2,df_final3,df_final4])
# I probably am doing something wrong here, I got all the tables to have one row representing one student on one module
# then added all the modules together

In [19]:
proj.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,sum_click
0,0,1,59,0,0,0,7,0,0,4,0,3,885
2,0,1,102,1,8,2,7,1,0,4,0,3,2236
4,0,1,116,1,8,1,8,1,0,4,0,3,880
5,0,1,160,1,11,0,5,0,0,4,0,3,2798
7,0,1,173,0,7,2,9,1,0,4,1,3,1923


In [20]:
proj2=proj.drop(columns="sum_click")

In [22]:
# Decision Tree for Table with sum_click

X = proj.loc[:, proj.columns != 'final_result']
y = proj['final_result']
xTrain, xTest, yTrain, yTest = train_test_split(X, y,train_size = 0.75)

dt = tree.DecisionTreeClassifier(criterion='gini')
dt = dt.fit(xTrain, yTrain)
train_pred = dt.predict(xTrain)
test_pred = dt.predict(xTest)
print("Accuracy:{0:.3f}".format(metrics.accuracy_score(yTest, test_pred)),"\n")

Accuracy:0.721 



C:\Users\jeffr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [23]:
# Decision Tree for Table without sum_click

X = proj2.loc[:, proj2.columns != 'final_result']
y = proj2['final_result']
xTrain, xTest, yTrain, yTest = train_test_split(X, y,train_size = 0.75)

dt = tree.DecisionTreeClassifier(criterion='gini')
dt = dt.fit(xTrain, yTrain)
train_pred = dt.predict(xTrain)
test_pred = dt.predict(xTest)
print("Accuracy:{0:.3f}".format(metrics.accuracy_score(yTest, test_pred)),"\n")

Accuracy:0.683 



In [25]:
# Random Forest for Table without sum_click
X = proj2.loc[:, proj2.columns != 'final_result']
y = proj2['final_result']
xTrain, xTest, yTrain, yTest = train_test_split(X, y,train_size = 0.75)

rf = RandomForestClassifier(n_estimators=10,random_state=33)
rf = rf.fit(xTrain, yTrain)
train_pred = rf.predict(xTrain)
test_pred = rf.predict(xTest)
print("Accuracy:{0:.3f}".format(metrics.accuracy_score(yTest, test_pred)),"\n")

Accuracy:0.742 



In [ ]:
# Random Forest for Table with sum_click
X = proj.loc[:, proj.columns != 'final_result']
y = proj['final_result']
xTrain, xTest, yTrain, yTest = train_test_split(X, y,train_size = 0.75)

rf = RandomForestClassifier(n_estimators=10,random_state=33)
rf = rf.fit(xTrain, yTrain)
train_pred = rf.predict(xTrain)
test_pred = rf.predict(xTest)
print("Accuracy:{0:.3f}".format(metrics.accuracy_score(yTest, test_pred)),"\n")